In [ ]:
from dataclasses import dataclass, field
import numpy as np
import numpy.typing as npt
from scipy.fftpack import fft, ifft, fftshift, ifftshift, fftfreq
from scipy.constants import pi, c, h
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy.signal import correlate
from scipy.optimize import curve_fit

rcParams['figure.dpi'] = 300
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['lines.linewidth'] = 3










def get_freq_range_from_time(time_s: npt.NDArray[float]
                             ) -> npt.NDArray[float]:
    """
    Calculate frequency range for spectrum based on time basis.

    When plotting a discretized pulse signal as a function of time,
    a time range is needed. To plot the spectrum of the pulse, one
    can compute the FFT and plot it versus the frequency range
    calculated by this function

    Parameters
    ----------
    time_s : npt.NDArray[float]
        Time range in seconds.

    Returns
    -------
    npt.NDArray[float]
        Frequency range in Hz.

    """
    return fftshift(fftfreq(len(time_s), d=time_s[1] - time_s[0]))


def get_time_from_freq_range(frequency_Hz: npt.NDArray[float]
                             ) -> npt.NDArray[float]:
    """
    Calculate time range for pulse based on frequency range.

    Essentially the inverse of the get_freq_range_from_time function.
    If we have a frequency range and take the iFFT of a spectrum field
    to get the pulse field in the time domain, this function provides the
    appropriate time range.

    Parameters
    ----------
    frequency_Hz : npt.NDArray[float]
        Freq. range in Hz.

    Returns
    -------
    time_s : npt.NDArray[float]
        Time range in s.

    """

    time_s = fftshift(fftfreq(len(frequency_Hz),
                              d=frequency_Hz[1] - frequency_Hz[0]))
    return time_s


def get_spectrum_from_pulse(time_s: npt.NDArray[float],
                            pulse_field: npt.NDArray[complex],
                            FFT_tol: float = 1e-3) -> npt.NDArray[complex]:
    """

    Parameters
    ----------
    time_s : npt.NDArray[float]
        Time range in seconds.
    pulse_field: npt.NDArray[complex]
        Complex field of pulse in time domain in units of sqrt(W).
    FFT_tol : float, optional
        When computing the FFT and going from temporal to spectral domain, the
        energy (which theoretically should be conserved) cannot change
        fractionally by more than FFT_tol. The default is 1e-7.

    Returns
    -------
    spectrum_field : npt.NDArray[complex]
        Complex spectral field in units of sqrt(J/Hz).

    """

    pulseEnergy = get_energy(time_s, pulse_field)  # Get pulse energy
    f = get_freq_range_from_time(time_s)
    dt = time_s[1] - time_s[0]

    assert dt > 0, (f"ERROR: dt must be positive, "
                    f"but {dt=}. {time_s[1]=},{time_s[0]=}")
    spectrum_field = ifftshift(
        ifft(ifftshift(pulse_field))) * (dt*len(f))  # Do shift and take fft
    spectrumEnergy = get_energy(f, spectrum_field)  # Get spectrum energy

    err = np.abs((pulseEnergy / spectrumEnergy - 1))


    assert (
        err < FFT_tol
    ), (f"ERROR = {err:.3e} > {FFT_tol:.3e} = FFT_tol : Energy changed "
        "when going from Pulse to Spectrum!!!")

    return spectrum_field


def get_pulse_from_spectrum(frequency_Hz: npt.NDArray[float],
                            spectrum_field: npt.NDArray[complex],
                            FFT_tol: float = 1e-3) -> npt.NDArray[complex]:
    """
    Converts the spectral field of a signal in the freq. domain temporal
    field in time domain

    Uses the iFFT to shift from freq. to time domain and ensures that energy
    is conserved

    Parameters
    ----------
    frequency_Hz : npt.NDArray[float]
        Frequency in Hz.
    spectrum_field : npt.NDArray[complex]
        Spectral field in sqrt(J/Hz).
    FFT_tol : float, optional
        Maximum fractional change in signal
        energy when doing FFT. The default is 1e-7.

    Returns
    -------
    pulse : npt.NDArray[complex]
        Temporal field in sqrt(W).

    """

    spectrumEnergy = get_energy(frequency_Hz, spectrum_field)

    time = get_time_from_freq_range(frequency_Hz)
    dt = time[1] - time[0]

    pulse = fftshift(fft(fftshift(spectrum_field))) / (dt*len(time))
    pulseEnergy = get_energy(time, pulse)

    err = np.abs((pulseEnergy / spectrumEnergy - 1))



    assert (
        err < FFT_tol
    ), (f"ERROR = {err:.3e} > {FFT_tol:.3e} = FFT_tol : Energy changed too "
        "much when going from Spectrum to Pulse!!!")
    return pulse


def get_energy(
    time_or_freq: npt.NDArray[float],
    field_in_time_or_freq_domain: npt.NDArray[complex],
) -> float:
    """
    Computes energy of signal or spectrum

    Gets the power or PSD of the signal from
    get_power(field_in_time_or_freq_domain)
    and integrates it w.r.t. either time or
    frequency to get the energy.

    Parameters
    ----------
    time_or_freq : npt.NDArray[float]
        Time range in seconds or freq. range in Hz.
    field_in_time_or_freq_domain : npt.NDArray[complex]
        Temporal field in [sqrt(W)] or spectral field [sqrt(J/Hz)].

    Returns
    -------
    energy: float
        Signal energy in J .

    """
    energy = np.trapz(
        get_power(field_in_time_or_freq_domain), time_or_freq)
    return energy

def get_power(field_in_time_or_freq_domain: npt.NDArray[complex]
              ) -> npt.NDArray[float]:
    """
    Computes temporal power or PSD

    For a real electric field, power averaged over an optical cycle is

    P = 1/T int_0^T( E_real**2 )dt.

    For a complex electric field, this same power is calculated as

    P = 0.5*|E|**2.

    Using the complex field makes calculations easier and the factor of
    0.5 is simply absorbed into the nonlinear parameter, gamma.
    Same thing works in the frequency domain.

    Parameters
    ----------
    field_in_time_or_freq_domain : npt.NDArray[complex]
        Temporal or spectral field.

    Returns
    -------
    power : npt.NDArray[complex]
        Temporal power (W) or PSD (J/Hz) at any instance or frequency.

    """
    power = np.abs(field_in_time_or_freq_domain) ** 2
    return power

def get_average(time_or_freq: npt.NDArray[float],
                pulse_or_spectrum: npt.NDArray[complex]) -> float:
    """
    Computes central time (or frequency) of pulse (spectrum)

    Computes central time (or frequency) of pulse (spectrum) by calculating
    the 'expectation value'.

    Parameters
    ----------
    time_or_freq : npt.NDArray[float]
        Time range in seconds or freq. range in Hz.
    pulse_or_spectrum : npt.NDArray[complex]
        Temporal or spectral field.

    Returns
    -------
    meanValue : float
        average time or frequency.

    """

    E = get_energy(time_or_freq, pulse_or_spectrum)
    meanValue = np.trapz(
        time_or_freq * get_power(pulse_or_spectrum), time_or_freq) / E
    return meanValue


def get_variance(time_or_freq: npt.NDArray[float],
                 pulse_or_spectrum: npt.NDArray[complex]) -> float:
    """
    Computes variance of pulse or spectrum

    Computes variance of pulse in time domain or freq domain via
    <x**2>-<x>**2

    Parameters
    ----------
    time_or_freq : npt.NDArray[float]
        Time range in seconds or freq. range in Hz.
    pulse_or_spectrum : npt.NDArray[complex]
        Temporal or spectral field.

    Returns
    -------
    variance : float
        variance in time or frequency domains.

    """
    E = get_energy(time_or_freq, pulse_or_spectrum)
    variance = (
        np.trapz(time_or_freq ** 2 *
                 get_power(pulse_or_spectrum), time_or_freq) / E
        - (get_average(time_or_freq, pulse_or_spectrum)) ** 2
    )
    return variance


def get_stdev(time_or_freq: npt.NDArray[float],
              pulse_or_spectrum: npt.NDArray[complex]) -> float:
    """
    Computes standard deviation of pulse or spectrum

    Computes std of pulse in time domain or freq domain via
    sqrt(<x**2>-<x>**2)

    Parameters
    ----------
    time_or_freq : npt.NDArray[float]
        Time range in seconds or freq. range in Hz.
    pulse_or_spectrum : npt.NDArray[complex]
        Temporal or spectral field.

    Returns
    -------
    stdev : float
        Stdev in time or frequency domains.

    """

    stdev = np.sqrt(get_variance(time_or_freq, pulse_or_spectrum))
    return stdev

def get_phase(pulse: npt.NDArray[complex]) -> npt.NDArray[float]:
    """
    Gets the phase of the pulse from its complex angle

    Calcualte phase by getting the complex angle of the pulse,
    unwrapping it and centering on middle entry.

    Parameters
    ----------
    pulse : npt.NDArray[complex]
        Complex electric field envelope in time domain.

    Returns
    -------
    phi : npt.NDArray[float]
        Phase of pulse at every instance in radians.

    """

    phi = np.unwrap(np.angle(pulse))  # Get phase starting from 1st entry
    phi = phi - phi[int(len(phi) / 2)]  # Center phase on middle entry
    return phi



In [ ]:
#Set up simulation parameters
dt_s = 0.1e-12 #Time resolution
N=10000 #Number of time steps = 2N+1
t_s = np.arange(-N,N+1,1)*dt_s #Time range
f_Hz=get_freq_range_from_time(t_s) #Frequency range


#Baseline CW field envelope is constant
field = np.ones_like(t_s)*1.0

#Set up random phase walk
step_size = 2*pi/(45)    #Moderate phase noise


#Generate several random walks 
N_walks = 100
random_phase_walk_list=np.zeros((N_walks,len(t_s)))
random_phase_walk_list_SHG=np.zeros((N_walks,len(t_s)))

phase_noise_power_spectrum_list=np.zeros_like(random_phase_walk_list)
phase_noise_power_spectrum_SHG_list=np.zeros_like(random_phase_walk_list)

average_spectrum = np.zeros_like(field)*1.0
average_spectrum_SHG = np.zeros_like(field)*1.0


for idx in range(N_walks):
    random_phase_walk = np.cumsum(np.random.choice([-1,1],len(t_s)))*step_size
    random_phase_walk -= random_phase_walk[N]
    random_phase_walk_exp = np.exp(1j*random_phase_walk)
    random_phase_walk_exp_SHG = random_phase_walk_exp**2

    power_spectrum = get_power( get_spectrum_from_pulse(t_s,random_phase_walk_exp)  )
    power_spectrum_SHG = get_power( get_spectrum_from_pulse(t_s,random_phase_walk_exp_SHG)  )

    random_phase_walk_list[idx,:]=random_phase_walk
    random_phase_walk_list_SHG[idx,:]=2*random_phase_walk 

    phase_noise_power_spectrum_list[idx,:]=power_spectrum
    phase_noise_power_spectrum_SHG_list[idx,:]=power_spectrum_SHG
    
    average_spectrum+=power_spectrum
    average_spectrum_SHG+=power_spectrum_SHG

average_spectrum/=N_walks
average_spectrum_SHG/=N_walks



In [ ]:


#Plot 3 particular random walks
fig, ax = plt.subplots()
ax.set_title("Three random phase walks")
ax.plot(t_s/1e-12,random_phase_walk_list[0],color='C7',alpha=0.7)
ax.plot(t_s/1e-12,random_phase_walk_list[int(N_walks/2)],color='k',alpha=0.7)
ax.plot(t_s/1e-12,random_phase_walk_list[N_walks-1],color='tab:brown',alpha=0.7)
ax.set_xlabel('Time [ps]')
ax.set_ylabel('$\phi(t)-\phi(0)$ [rad]')   
ax.set_ylim(-50,50)
plt.show()
 
#Plot all random walks simultaneously to illustrate "Gaussian" broadening.
start_idx = int(0.5*len(t_s)+1)
middle_idx=int(0.75*len(t_s))
late_idx=len(t_s)-1

fig, ax = plt.subplots()
ax.set_title(f"{N_walks} Phase Evolutions")
ax.axvline(x=t_s[start_idx]/1e-12,color='C0',alpha=0.5)
ax.axvline(x=t_s[middle_idx]/1e-12,color='C1',alpha=0.5)
ax.axvline(x=t_s[late_idx]/1e-12,color='C2',alpha=0.5)
for idx in range(0,N_walks):
    ax.plot(t_s/1e-12,random_phase_walk_list[idx,:],color='C7',alpha=5/N_walks)
ax.set_xlabel('Time [ps]')
ax.set_ylabel('$\phi(t)-\phi(0)$ [rad]')   
ax.set_ylim(-50,50)
plt.show()



def gauss(t,sigma):
    return 1/np.sqrt(2*pi*sigma**2)*np.exp(-0.5*(t/sigma)**2)

early_list=random_phase_walk_list[:,start_idx]
middle_list=random_phase_walk_list[:,middle_idx]
late_list=random_phase_walk_list[:,late_idx]

bins=np.linspace(-np.max( np.abs(late_list)),np.max( np.abs(late_list)),30)
xplot=np.linspace(-np.max(bins),np.max(bins),1000)
fig,ax=plt.subplots()
ax.set_title(f'Histograms of {N_walks} phase values')
ax.hist(early_list,bins=bins,alpha=0.5,label='Early in walk',density=True)
ax.hist(middle_list,bins=bins,alpha=0.5,label='Middle of walk',density=True)
ax.hist(late_list,bins=bins,alpha=0.5,label='End of walk',density=True)

#Note that we plot the Gaussians using the theoretical STDEV = sqrt(N_steps_so_far)*step_size for a 1D random walk
ax.plot(xplot,gauss(xplot,1*step_size),color='C0',alpha=0.5)
ax.plot(xplot,gauss(xplot,np.sqrt(N/2)*step_size),color='C1',alpha=0.5)
ax.plot(xplot,gauss(xplot,np.sqrt(N)*step_size),color='C2',alpha=0.5)

ax.set_ylim(3e-3,0.5)
ax.set_xlabel('$\phi(t)-\phi(0)$ [rad]')
ax.legend()
ax.set_yscale('log')
plt.show()







In [ ]:
#Plot power spectrum of one particular random walk




fig, ax = plt.subplots()
ax.set_title("$|FT[E(t)]|^2$")
ax.plot(f_Hz/1e12,power_spectrum,color='C7',alpha=0.7)
Gamma=step_size**2/(2*np.pi*dt_s)
df=step_size**2/(2*np.pi*dt_s)
lor = Gamma/( (Gamma/2)**2+(f_Hz)**2)/2/np.pi/(f_Hz[1]-f_Hz[0])
#ax.plot(f_Hz/1e12, lor    ,color='r',alpha=0.7)
ax.set_xlabel('Freq [THz]')
ax.set_ylabel('Spectrum [norm.]')
ax.set_xlim(-2*df/1e12,2*df/1e12)
ax.grid()
plt.show()

fig, ax = plt.subplots()
ax.set_title("$|FT[E(t)]|^2$")
ax.plot(f_Hz/1e12,power_spectrum,color='C7',alpha=0.7)
Gamma=step_size**2/(2*np.pi*dt_s)
df=step_size**2/(2*np.pi*dt_s)
lor = Gamma/( (Gamma/2)**2+(f_Hz)**2)/2/np.pi/(f_Hz[1]-f_Hz[0])
#ax.plot(f_Hz/1e12, lor    ,color='r',alpha=0.7)
ax.set_xlabel('Freq [THz]')
ax.set_ylabel('Spectrum [norm.]')
ax.set_xlim(-2*df/1e12,2*df/1e12)
ax.set_yscale('log')
ax.set_ylim(np.max(power_spectrum)*1e-3,np.max(power_spectrum)*3)
ax.grid()
plt.show()



In [ ]:
#Plot average spectrum and theoretical average spectrum (Lorentzian) 
#according to the Wiener-Khinchin Theorem    

fig, ax = plt.subplots()
ax.set_title("$<|FT[E(t)]|^2>$")
ax.plot(f_Hz/1e12,average_spectrum,color='C7',alpha=0.7)
Gamma=step_size**2/(2*np.pi*dt_s)
lor = Gamma/( (Gamma/2)**2+(f_Hz)**2)/2/np.pi/(f_Hz[1]-f_Hz[0])
ax.plot(f_Hz/1e12, lor    ,color='k',alpha=0.7)
ax.set_xlabel('Freq [THz]')
ax.set_ylabel('Spectrum [norm.]')
ax.set_xlim(-2*df/1e12,2*df/1e12)
ax.grid()
plt.show()

#Same plot as above on a log scale
fig, ax = plt.subplots()
ax.set_title("$<|FT[E(t)]|^2>$")
ax.plot(f_Hz/1e12,average_spectrum,color='C7',alpha=0.7)
df=step_size**2/(2*np.pi*dt_s)
lor = df/( (df/2)**2+(f_Hz)**2)/2/np.pi/(f_Hz[1]-f_Hz[0])
ax.plot(f_Hz/1e12, lor    ,color='k',alpha=0.7)
ax.set_xlabel('Freq [THz]')
ax.set_ylabel('Spectrum [norm.]')
ax.set_yscale('log')
ax.set_xlim(-10*df/1e12,10*df/1e12)
ax.set_ylim(np.max(lor)*1e-3,np.max(lor)*3)
ax.grid()
plt.show()

In [ ]:
#Now repeat for the SHG case

#Plot random walks for input and SHG
fig, ax = plt.subplots()
ax.set_title("Random phase walks for input and SHG")
ax.plot(t_s/1e-12,random_phase_walk_list[0],color='C7',alpha=0.7,label='Input')
ax.plot(t_s/1e-12,random_phase_walk_list_SHG[0],color='C8',alpha=0.7,label='SHG')
ax.set_xlabel('Time [ps]')
ax.set_ylabel('$\phi(t)-\phi(0)$ [rad]')   
ax.set_ylim(-70,70)
ax.legend()
plt.show()




In [ ]:
#Plot all random walks simultaneously to illustrate "Gaussian" broadening.
start_idx = int(0.5*len(t_s)+1)
middle_idx=int(0.75*len(t_s))
late_idx=len(t_s)-1

fig, (ax1,ax2) = plt.subplots(1, 2)
ax1.set_title(f"Input {N_walks} Phase Evolutions")
ax2.set_title(f"SHG {N_walks} Phase Evolutions")

ax1.axvline(x=t_s[late_idx]/1e-12,color='C2',alpha=0.5)
ax2.axvline(x=t_s[late_idx]/1e-12,color='C3',alpha=0.5)

for idx in range(0,N_walks):
    ax1.plot(t_s/1e-12,random_phase_walk_list[idx,:],color='C7',alpha=5/N_walks)
    ax2.plot(t_s/1e-12,random_phase_walk_list_SHG[idx,:],color='C8',alpha=5/N_walks)
ax1.set_xlabel('Time [ps]')
ax1.set_ylabel('$\phi(t)-\phi(0)$ [rad]')   
ax1.set_ylim(-70,70)
ax2.set_xlabel('Time [ps]')
ax2.set_ylim(-70,70)
plt.show()


In [ ]:

late_list=random_phase_walk_list[:,late_idx]
late_list_SHG=random_phase_walk_list_SHG[:,late_idx]

bins=np.linspace(-np.max( np.abs(late_list_SHG)),np.max( np.abs(late_list_SHG)),30)
xplot=np.linspace(-np.max(bins),np.max(bins),1000)
fig,ax=plt.subplots()
ax.set_title(f'Histograms of {N_walks} phase values')

ax.hist(late_list,bins=bins,alpha=0.5,color='C2',label='End of walk, Input',density=True)
ax.hist(late_list_SHG,bins=bins,alpha=0.5,color='C3',label='End of walk, SHG',density=True)

#Note that we plot the Gaussians using the theoretical STDEV = sqrt(N_steps_so_far)*step_size for a 1D random walk
ax.plot(xplot,gauss(xplot,np.sqrt(N)*step_size),color='C2',alpha=0.5)
ax.plot(xplot,gauss(xplot,np.sqrt(N)*2*step_size),color='C3',alpha=0.5) #Note 2x stdev = 4x variance for SHG

ax.set_ylim(3e-3,0.5)

ax.legend()
ax.set_yscale('log')
plt.show()


In [ ]:
#Plot average spectrum and theoretical average spectrum (Lorentzian) 
#according to the Wiener-Khinchin Theorem    

fig, ax = plt.subplots()
ax.set_title("$<|FT[E(t)]|^2>$")
ax.plot(f_Hz/1e12,average_spectrum,color='C7',alpha=0.7)
ax.plot(f_Hz/1e12,average_spectrum_SHG,color='C8',alpha=0.7)

Gamma=step_size**2/(2*np.pi*dt_s)
Gamma_SHG=4*Gamma

lor = Gamma/( (Gamma/2)**2+(f_Hz)**2)/2/np.pi/(f_Hz[1]-f_Hz[0])
lor_SHG = Gamma_SHG/( (Gamma_SHG/2)**2+(f_Hz)**2)/2/np.pi/(f_Hz[1]-f_Hz[0])

ax.plot(f_Hz/1e12, lor    ,color='k',alpha=0.7)
ax.plot(f_Hz/1e12, lor_SHG    ,color='b',alpha=0.7)

ax.set_xlabel('Freq [THz]')
ax.set_ylabel('Spectrum [norm.]')
ax.set_xlim(-2*df/1e12,2*df/1e12)
ax.grid()
plt.show()

fig, ax = plt.subplots()
ax.set_title("$<|FT[E(t)]|^2>$")
ax.plot(f_Hz/1e12,average_spectrum,color='C7',alpha=0.7)
ax.plot(f_Hz/1e12,average_spectrum_SHG,color='C8',alpha=0.7)
ax.plot(f_Hz/1e12, lor    ,color='k',alpha=0.7)
ax.plot(f_Hz/1e12, lor_SHG    ,color='b',alpha=0.7)
ax.set_xlabel('Freq [THz]')
ax.set_ylabel('Spectrum [norm.]')
ax.set_yscale('log')
ax.set_xlim(-10*df/1e12,10*df/1e12)
ax.set_ylim(np.max(lor)*1e-3,np.max(lor)*3)
ax.grid()
plt.show()
